In [1]:
import os
import sys
import json
import openai
from dotenv import load_dotenv

import psycopg
from psycopg.rows import dict_row

from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, AIMessagePromptTemplate,PromptTemplate
from langchain.chains import ConversationChain, LLMChain
from langchain.schema.output_parser import StrOutputParser

from langchain.agents.agent_types import AgentType
from langchain.agents import initialize_agent
from langchain.memory import ConversationBufferMemory,ConversationSummaryBufferMemory,ConversationSummaryMemory,ReadOnlySharedMemory
from langchain.schema import HumanMessage, AIMessage


from datetime import datetime
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import BaseMessage,message_to_dict,messages_from_dict

from typing import List

OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')
db_config = {
    'dbname': os.environ.get('db_name'),
    'dbuser': os.environ.get('db_username'),
    'dbpassword': os.environ.get('db_password'),
    'dbhost': os.environ.get('db_host'),
    'dbport': os.environ.get('db_port'),
}

llms = ChatOpenAI(model="gpt-4-1106-preview", temperature=0.9, api_key=OPENAI_API_KEY)

In [2]:
class PostGresMemoryHistory(BaseChatMessageHistory):
    def __init__(self, db_name:str, db_user:str, db_password:str, db_host:str, db_port:str, table_name:str, chat_id:str):
        self.db_config = {
            'dbname': db_name,     
            'user': db_user,        
            'password': db_password,    
            'host': db_host,           
            'port': db_port,
            'table_name': table_name,                 
        }
        
        self.chat_id = chat_id
        
        
        try:
            self.conn = psycopg.connect(dbname=self.db_config['dbname'], user=self.db_config['user'], password=self.db_config['password'], host=self.db_config['host'], port=self.db_config['port'])
            self.cur = self.conn.cursor(row_factory=dict_row)
            self.create_table_if_not_exists()
        except psycopg.Error as e:
            print("Error connecting to PostgreSQL database:", e)
            self.conn = None
            self.cur = None
                  
    
    def create_table_if_not_exists(self):
        create_table_query = f"""CREATE TABLE IF NOT EXISTS {self.db_config['table_name']} (
            id SERIAL PRIMARY KEY,
            chat_id TEXT NOT NULL,
            message JSONB NOT NULL, 
            time_stamp TIMESTAMP NOT NULL
        );"""
        self.cur.execute(create_table_query)
        self.conn.commit()
    
    def add_ai_message(self, message: str) -> None:
        return self.add_message(AIMessage(content=message))
    
    def add_user_message(self, message: str) -> None:
        return self.add_message(HumanMessage(content=message))
    
    @property
    def messages(self) -> List[BaseMessage]:  # type: ignore
        """Retrieve the messages from PostgreSQL"""
        query = (
            f"SELECT * FROM {self.db_config['table_name']} WHERE chat_id = %s ORDER BY time_stamp;"
        )
        self.cur.execute(query, (self.chat_id,))
        items = [record["message"] for record in self.cur.fetchall()]
        messages = messages_from_dict(items)
        return messages
    
    
    def add_message(self, message:BaseMessage):
        time_stamp = datetime.now()
        query = f"INSERT INTO {self.db_config['table_name']} (chat_id, message, time_stamp) VALUES (%s, %s, %s);"
        self.cur.execute(query, (self.chat_id, json.dumps(message_to_dict(message)), time_stamp))
        self.conn.commit()
        
            
    def clear(self) -> None:
        conn = psycopg.connect(self.db_config)
        cur = conn.cursor()
        if conn:
            try:
                query = f"DELETE FROM {self.db_config['dbname']} WHERE session_id = %s;"
                cur.execute(query, (self.session_id,))
                conn.commit()
                cur.close()
                conn.close()
            except psycopg.Error as e:
                print("Error clearing DB:", e)
        else:
            print("Failed to establish connection...")

In [3]:
template = """
The following is a conversation with an AI assistant and Human. The assistant is helpful, creative and very friendly. The assistant is also a great prompt engineer and AI developer.
AI is helping human to solve AI problems regarding Contextual Learning in Large Language Models (LLM) by optimizing memory.

Current Conversation History : {history}
Human: {input}
AI : 

"""

In [4]:
conversation_template = PromptTemplate(template=template, input_variables=["history", "input"])

In [5]:
chat_id = '485276d7-8dab-48b1-a8ba-ad65a7108090'

In [6]:
postgres_memory = PostGresMemoryHistory(db_host=os.environ.get('db_host'), db_name=os.environ.get('db_name'), db_user=os.environ.get('db_username'), db_password=os.environ.get('db_password'), db_port=os.environ.get('db_port'), table_name=os.environ.get('conversation_table_name'), chat_id=chat_id)
memory  = ConversationSummaryMemory.from_messages(chat_memory=postgres_memory, llm=llms, return_messages=True,verbose=True)

In [8]:
conversation_bufw = ConversationChain(
    llm=llms, 
    memory=memory,
    prompt=conversation_template,
)

In [39]:
input = """Engineer for Continual Learning in LLM agents
Testing task

Objective: The goal of this task is to test your understanding of using LLMs and inventiveness through adding a prototype continual learning system to a conversational LLM agent.  

Task Description:  Context length is an important factor in the learning of Large Language Models. While contexts are getting longer and longer, there has been work (https://arxiv.org/abs//2307.03172) which suggests that LLMs can fail at performing an information dependent task, even if the LLM has all of the information that it needs in its context.  

This suggests that there is a need for continual learning systems. These systems augment LLMs by giving the LLM the ability to remember instructions, interactions, and relevant results from its conversations with others. For this task, you will produce a system that can resolve four issues that occur in continual learning:
Duplicated memories: If there are repeated events, you don’t want to recall all of them. Otherwise you may fill up the context with identical events and retrievals will be uninformative.
Contradictory memories: If there are two instructions that contradict each other, how should the LLM respond when it should execute one of the instructions? 
Temporary memories: Information with some finite lifetime (E.g “Until further notice…”)
Episodic memories: refer to the ability of the LLM agent to remember specific events or interactions in a time-stamped manner. Unlike general rules or guidelines, these are memories tied to particular instances or conversations, allowing the LLM agent to reference past interactions for context or to provide more personalized and informed responses.

This task involves using examples of four different types of information to create a solution. The key points to consider are:

Utilize Provided Examples: The task comes with examples for each information type. These should be used as a guide in developing your solution.

Avoid Over-Reliance on LLM Context: While the LLM has the capability to store and process information within its context, it's important not to depend too much on this feature. The assumption is that the agent (LLM) will engage in lengthy and iterative conversations over hours or days, which means it cannot retain all relevant details in its immediate context due to space limitations.

Recommended LLM: It is advised to use GPT-4 for this task, considering its advanced ability to follow instructions accurately. The context size should be capped at 8,000 tokens. This limitation is intentional to ensure the necessity of an external long-term memory system, as the 8K context size is insufficient for storing all details from prolonged conversations.


Technical Requirements:
Conversational LLM agents augmented with a continual learning system
Given the rich ecosystem of AI technologies that are supported, for this task you should use Python.
Feel free to utilize any libraries or frameworks, including third-party or paid ones. The LLM that you use is anticipated to impact the quality of the continual learning, but we will also test your solution using a range of LLMs. We will weigh the approach and inventiveness of the solution higher than the LLM-dependent performance of the solution.
Focus on creating a simple but functional prototype rather than a polished final product.

We will evaluate your task based on:
Functionality of the system.
How well the system deals with the four types of information
Efficiency of the implementation.
Code quality and organization.
How effectively you utilized third-party libraries or services to achieve the task.

Please submit your code along with a brief report describing your design choices, the LLM that you used for development, any challenges you faced, how you overcame them, and a justification for the third-party tools used. Include instructions on how to run the code and any dependencies needed.

This task is intended to give us an idea of your skill set, creativity, and your ability to effectively leverage existing technologies. We look forward to seeing your innovative solutions!

Examples of information types:
Duplicated information:

User: I am going outside.
Agent: Have a good time outside!
…
User: I have returned home and am going to take a bath
Agent: Enjoy your bath!
…
User: I am going outside again.
Agent: Thanks for letting me know, enjoy your time outside.

Note: We expect to see that the memories for the user going outside are consolidated in some fashion (e.g combining both, or deleting one of the memories) so that after a large number of repetitions of an event, the memory retrieval only contains one instance of “going outside”

Contradictory Information:

User: Whenever I go out to my car, remind me to fill up the screenwash.
Agent: Okay, I will remind you to fill the screenwash when you go out to your car.

User: Oh, also remind me to check the oil when I go out to the car.
Agent: Okay, I will remind you to fill your screenwash and check your oil when you go out to your car.
…
User: I have filled my screenwash and checked my oil. Next time I go out to my car, remind me to check my tires.
Agent: I will remind you to check your tires the next time you go out to your car.

Note: There have been three directives for reminders. The second is additive to the first, but the third one contradicts the first two because the user has performed the original tasks. If all three of these events are in the LTM, which one would be chosen? We do not want the agent to remind the user about the screenwash and oil, but only the tires. 


Not Contradictory information:

User: Hey, can you set up a reminder for me?
Agent: Sure, what would you like to be reminded of?

User: I want to be reminded to leave the house at 2pm today.
Agent: Alright, I've set a reminder for you to leave the house at 2pm today. Anything else you need?

User: Will you remind me again, when I have to leave the house at 6pm this evening?
Agent: Absolutely, I've also set a reminder for you to leave the house at 6pm today. 

Temporary Information:
User: Remember this shopping list for me: Bread, Eggs, and Milk
Agent: I have remembered your shopping list of Bread, Eggs, and Milk

User: I have bought some eggs.
Agent: Okay, I have removed the Eggs from your shopping list.
…
User: What is my shopping list again?
Agent: On your shopping list you have: Bread and Milk

Note: A shopping list is piece of information that is very volatile, so the agent and its memory should be reactive to additions and deletions, returning only the latest version of the shopping list when asked.

—

User: When I return home from work, suggest a meal I can make for dinner.
Agent: I will remember to suggest a meal that you can make for dinner when you get in from work.
…
User: I have preplanned meals this week, so for this week, don’t make any suggestions about what to make for dinner when I get home from work.
Agent: All right, for this week, I will not suggest meals for you when you return from work. 
…
User: I have finished my preplanned meals, go back to suggesting meals for me.
Agent: I will resume suggesting meals for you when you return from work

Note: We have given the agent some directive in the past, then given it a contradictory instruction with an explicit understanding that this instruction will be temporary. Have the LTM deal with this temporary change of the instructions so that the agent can achieve the temporary task, and then go back to original instructions.



Episodic Memory
Two days ago:
User: I have bought a side dresser for the bedroom, and some ingredients for dinner tonight. I am going to make omelets!
Agent: That’s great, you needed a side dresser for your bed, and omelets are a healthy dinner!
Today:
User: I think I will prepare something for dinner tonight… Do you think I could use the ingredients I bought two days ago?
Agent: The eggs should be fine, unless you actually made and ate the omelets then!

—

Yesterday:
User: I have purchased some items for the house: A mirror, two potted plants and a standing lamp.
Agent: Okay, hope you enjoy decorating your home!
…
Today:
User: Yesterday I got home with some new items for the house. Do you remember what they were?
Agent: Sure! You bought A mirror, two potted plants and a standing lamp.


Note: Episodic retrieval allows the agent to order its memories through time, and recall or filter them using timestamps to answer questions with a time based component. 

"""

In [40]:
conversation_bufw.run(input = input)

"As an AI language model, I can guide you through the conceptual design of a prototype system for continual learning in LLMs, such as GPT-4, and address the points mentioned in your task description. Below, I outline a potential approach and considerations for each type of information.\n\n### Design Considerations:\n\n#### External Memory Store:\n- To overcome the limited context size of LLMs, we'll need an external memory store. This could be a NoSQL database, a key-value store, or any other efficient data structure that can handle the different types of memories we need to store, ensuring quick retrieval and updates.\n\n#### Memory Encoder:\n- Since we have a limit on the context size, we'll need to encode memories in a way that's information-dense but also easily decodable by the LLM. This could involve summarizing events or extracting key-value pairs.\n\n### Dealing with Different Information Types:\n\n#### Duplicated Information:\n- Use hashing or fingerprinting techniques to iden

In [41]:
result = conversation_bufw.run('The above input is the task description. Please read it carefully and try to understand it. I will ask you questions about it.')
print(result)

Understood. I've carefully read the task description. How can I assist you with questions regarding this project?


In [42]:
result = conversation_bufw.run('Before starting the task, I want to ask you a question. Do you know about langchain?')

In [43]:
print(result)

Yes, LangChain is a Python library that's designed to simplify the process of building applications with Large Language Models (LLMs) like GPT-3, GPT-4, and others. It provides a set of tools and interfaces that abstract away some of the complexities involved in creating AI-powered applications, making it easier to prototype, build, and scale applications that leverage language models.

Among its features, LangChain offers components for interacting with language models, chaining different AI operations together, managing state and context, and integrating external knowledge sources. LangChain can be especially useful for projects like the one we're discussing as it could potentially help manage the different types of memory you're looking to implement within your LLM agent. It might provide a structured way to handle the integration of continual learning and memory management within the context of a conversational agent.

If you are considering using LangChain for this project, it cou

In [44]:
result = conversation_bufw.run('Tell me your idead about managing  managing duplicated, contradictory, temporary, and episodic memories within the context constraints.')
print(result)

Managing different types of memories—duplicated, contradictory, temporary, and episodic—within the context constraints of a GPT-4 based conversational agent can indeed be challenging. Here are some ideas for handling each:

1. **Duplicated Memories:**
   - **Deduplication Process:** Implement a mechanism to check new information against the existing memory store to identify and merge duplicates. This could involve hashing memory fragments and comparing hashes before storage.
   - **Reference Counting:** Store memory fragments with a reference count that increments with each duplication, and use this count to prioritize which fragments to keep within the context window.

2. **Contradictory Memories:**
   - **Version Control:** Maintain a versioning system where newer information can overwrite older, contradictory memories, perhaps with user or system verification for critical contradictions.
   - **Confidence Scoring:** Assign confidence scores to each memory fragment based on source re

In [45]:
result = conversation_bufw.run('My Idea of solving this problem is to use langchain ConversationSummaryMemory. Get the conversation summary as the context.')
print(result)

Your idea to leverage LangChain's ConversationSummaryMemory component is a strategic approach for optimizing contextual learning in Large Language Models like GPT-4. This method can distill the essence of a conversation into a concise summary, which serves as an efficient context for the model to reference, thus preserving memory and processing bandwidth. Here's how we can elaborate on your idea:

1. **Integration**: We can integrate ConversationSummaryMemory into the conversational agent's architecture. This integration will allow the agent to automatically generate a summary after each interaction or after a predefined number of interaction turns, to keep the context relevant and up to date.

2. **Summary Generation**: The summaries should capture key points, questions, answers, and actionable items from the conversation. Natural language processing techniques, such as Named Entity Recognition (NER) and sentiment analysis, can be used to identify and highlight important aspects.

3. 

In [46]:
implimentation_process = """
1.I have created a postgres database where i store the conversation based on chat_id, message(borh human and ai), and time_stamp.I have created a class PostGresMemoryHistory to handle the postgres database.
2. I am using langchain ConversationSummaryMemory to get the conversation summary as the context from the postgres database based on chat_id.

Now the idea i am thinking is to build Custom Tool from LangChain Tools.
where i will build tools for managing duplicated, contradictory, temporary, and episodic memories within the chat constraints.
I will pass the database to all the types of memory and create a prompt which will guide agent to perform task and use the right memory tool for the task.

What are your thoughts  on this idea? Any suggestions?
"""

In [47]:
result = conversation_bufw.run(implimentation_process)
print(result)

Your approach to integrating LangChain's ConversationSummaryMemory with a Postgres database to manage conversation history categorized by chat_id, message contents, and time stamps is a solid foundational strategy for contextual learning in a Large Language Model (LLM) like GPT-4. Creating a custom tool from LangChain Tools that specifically addresses duplicated, contradictory, temporary, and episodic memories is an innovative proposal that shows promise for enhancing the conversational AI's performance.

Here are some thoughts and suggestions:

1. Automated Deduplication: For managing duplicates, ensure your tool has an efficient mechanism to recognize and merge similar memory entries or to flag and remove them, depending on the context requirement. This may involve natural language understanding algorithms that go beyond simple text matching.

2. Contradiction Resolution: Develop a method for the AI to determine the most reliable information when contradictions arise. This could invo

In [48]:
result = conversation_bufw.run('Can you help me desgining and implimenting the tool?')
print(result)

Absolutely, I'd be delighted to assist you in designing and implementing the tool for optimizing memory in Large Language Models. Here's how we can break down the project:

1. **Design Stage:**
   - **Define the Requirements:** List down all the functionalities you expect from the tool, such as handling duplicates, resolving contradictions, managing temporal relevance, and so on.
   - **Create a Workflow Diagram:** This will help us visualize how the tool will fit into the existing infrastructure and how it will manage memory.
   - **Choose Technologies:** Based on requirements, we can select suitable technologies. For example, we're considering a Postgres database, but we should also decide on the programming languages, frameworks, and libraries we will use.

2. **Implementation Stage:**
   - **Database Schema Design:** Design a well-structured schema for the Postgres database that will store conversation data efficiently.
   - **Develop the Tool:** Code the functionalities, starting 

In [52]:
conv_desgin = """
I have already created postgres database 

1,485276c7-8d9b-48b1-a8aa-ad65a7118192,"{""data"": {""type"": ""human"", ""content"": ""Hello, how are you?"", ""example"": false, ""additional_kwargs"": {}}, ""type"": ""human""}",2023-12-07 16:03:14.317455
2,485276c7-8d9b-48b1-a8aa-ad65a7118192,"{""data"": {""type"": ""ai"", ""content"": ""Hello, how are you?"", ""example"": false, ""additional_kwargs"": {}}, ""type"": ""ai""}",2023-12-07 16:03:14.324714
3,485276c7-8d9b-48b1-a8aa-ad65a7118192,"{""data"": {""type"": ""human"", ""content"": ""Can you tell me about yourself?"", ""example"": false, ""additional_kwargs"": {}}, ""type"": ""human""}",2023-12-07 16:03:17.474474
4,485276c7-8d9b-48b1-a8aa-ad65a7118192,"{""data"": {""type"": ""ai"", ""content"": ""Of course! I am an AI language model developed by OpenAI called GPT-3. I have been trained on a wide range of data sources, including books, articles, and websites, to generate human-like responses to text inputs. I can assist with various tasks, answer questions, engage in conversations, and even provide creative writing suggestions. Is there anything specific you would like to know about me?"", ""example"": false, ""additional_kwargs"": {}}, ""type"": ""ai""}",2023-12-07 16:03:17.476851
5,485276c7-8d9b-48b1-a8aa-ad65a7118192,"{""data"": {""type"": ""human"", ""content"": ""How AI can be used in healthcare?"", ""example"": false, ""additional_kwargs"": {}}, ""type"": ""human""}",2023-12-07 17:37:40.03489
6,485276c7-8d9b-48b1-a8aa-ad65a7118192,"{""data"": {""type"": ""ai"", ""content"": ""AI can be used in healthcare in a variety of ways. One of the main applications is in medical imaging, where AI algorithms can analyze images such as X-rays, CT scans, and MRIs to help detect and diagnose diseases. AI can also be used to predict patient outcomes and identify high-risk patients, allowing healthcare providers to intervene earlier and provide more personalized care. Additionally, AI can assist in drug discovery and development by analyzing large amounts of data and identifying potential drug candidates. AI-powered chatbots and virtual assistants can also be used to provide patient support and answer common medical questions. Overall, AI has the potential to greatly improve healthcare outcomes and efficiency."", ""example"": false, ""additional_kwargs"": {}}, ""type"": ""ai""}",2023-12-07 17:37:40.06402
7,485276c7-8d9b-48b1-a8aa-ad65a7118192,"{""data"": {""type"": ""human"", ""content"": ""And specifically, how will ai will be used to do that?"", ""example"": false, ""additional_kwargs"": {}}, ""type"": ""human""}",2023-12-07 17:39:48.345355
8,485276c7-8d9b-48b1-a8aa-ad65a7118192,"{""data"": {""type"": ""ai"", ""content"": ""AI can be used in healthcare in a variety of specific ways. For example, in analyzing medical images, AI algorithms can be trained to detect abnormalities or assist in diagnosing diseases such as cancer. This can help doctors make more accurate and timely diagnoses, leading to better patient outcomes.\n\nAI can also be used to predict patient outcomes. By analyzing large amounts of patient data, AI algorithms can identify patterns and make predictions about a patient's likelihood of developing certain conditions or experiencing certain outcomes. This can help doctors personalize treatment plans and interventions for individual patients, improving their overall care.\n\nIn drug discovery, AI can assist in the identification and development of new drugs. By analyzing vast amounts of data, including genetic information and molecular structures, AI algorithms can help researchers identify potential drug candidates and predict their effectiveness. This can accelerate the drug discovery process and lead to the development of more effective treatments.\n\nAI can also provide patient support. Virtual assistants powered by AI can answer patient questions, provide information about medications or treatment options, and even offer emotional support. This can help patients access the information and support they need, especially in situations where healthcare professionals may not be readily available.\n\nOverall, AI has the potential to greatly improve healthcare outcomes and efficiency by assisting healthcare professionals in various tasks, providing personalized care, and enabling faster and more accurate diagnoses and treatments."", ""example"": false, ""additional_kwargs"": {}}, ""type"": ""ai""}",2023-12-07 17:39:48.349051
9,485276c7-8d9b-48b1-a8aa-ad65a7118192,"{""data"": {""type"": ""human"", ""content"": ""Which Algorithm is will be used to detect abnormality in analyzing medical images"", ""example"": false, ""additional_kwargs"": {}}, ""type"": ""human""}",2023-12-07 17:45:20.467854
10,485276c7-8d9b-48b1-a8aa-ad65a7118192,"{""data"": {""type"": ""ai"", ""content"": ""There are several algorithms that can be used to detect abnormalities in medical images. One commonly used algorithm is called Convolutional Neural Network (CNN). CNNs are particularly effective in image analysis tasks because they can automatically learn and extract features from images. Another algorithm that is often used is called Support Vector Machines (SVM). SVMs are a type of supervised learning algorithm that can classify data into different categories based on the features extracted from the images. Additionally, there are other algorithms such as Random Forests, Decision Trees, and Deep Learning algorithms that can also be used for abnormality detection in medical images."", ""example"": false, ""additional_kwargs"": {}}, ""type"": ""ai""}",2023-12-07 17:45:20.475284


This is how the database looks like. I have created a class PostGresMemoryHistory to handle the postgres database.

class PostGresMemoryHistory(BaseChatMessageHistory):
    def __init__(self, db_name:str, db_user:str, db_password:str, db_host:str, db_port:str, table_name:str, chat_id:str):
        self.db_config = {
            'dbname': db_name,     
            'user': db_user,        
            'password': db_password,    
            'host': db_host,           
            'port': db_port,
            'table_name': table_name,                 
        }
        
        self.chat_id = chat_id
        
        
        try:
            self.conn = psycopg.connect(dbname=self.db_config['dbname'], user=self.db_config['user'], password=self.db_config['password'], host=self.db_config['host'], port=self.db_config['port'])
            self.cur = self.conn.cursor(row_factory=dict_row)
            self.create_table_if_not_exists()
        except psycopg.Error as e:
            print("Error connecting to PostgreSQL database:", e)
            self.conn = None
            self.cur = None
                  
    
    def create_table_if_not_exists(self):
        create_table_query = fCREATE TABLE IF NOT EXISTS {self.db_config['table_name']} (
            id SERIAL PRIMARY KEY,
            chat_id TEXT NOT NULL,
            message JSONB NOT NULL, 
            time_stamp TIMESTAMP NOT NULL
        );"
        self.cur.execute(create_table_query)
        self.conn.commit()
    
    def add_ai_message(self, message: str) -> None:
        return self.add_message(AIMessage(content=message))
    
    def add_user_message(self, message: str) -> None:
        return self.add_message(HumanMessage(content=message))
    
    @property
    def messages(self) -> List[BaseMessage]:  # type: ignore
        "Retrieve the messages from PostgreSQL"
        query = (
            f"SELECT * FROM {self.db_config['table_name']} WHERE chat_id = %s ORDER BY time_stamp;"
        )
        self.cur.execute(query, (self.chat_id,))
        items = [record["message"] for record in self.cur.fetchall()]
        messages = messages_from_dict(items)
        return messages
    
    
    def add_message(self, message:BaseMessage):
        time_stamp = datetime.now()
        query = f"INSERT INTO {self.db_config['table_name']} (chat_id, message, time_stamp) VALUES (%s, %s, %s);"
        self.cur.execute(query, (self.chat_id, json.dumps(message_to_dict(message)), time_stamp))
        self.conn.commit()
        
            
    def clear(self) -> None:
        conn = psycopg.connect(self.db_config)
        cur = conn.cursor()
        if conn:
            try:
                query = f"DELETE FROM {self.db_config['dbname']} WHERE session_id = %s;"
                cur.execute(query, (self.session_id,))
                conn.commit()
                cur.close()
                conn.close()
            except psycopg.Error as e:
                print("Error clearing DB:", e)
        else:
            print("Failed to establish connection...")
            
postgres_memory = PostGresMemoryHistory(db_host=os.environ.get('db_host'), db_name=os.environ.get('db_name'), db_user=os.environ.get('db_username'), db_password=os.environ.get('db_password'), db_port=os.environ.get('db_port'), table_name=os.environ.get('conversation_table_name'), chat_id=chat_id)
memory  = ConversationSummaryMemory.from_messages(chat_memory=postgres_memory, llm=llms, return_messages=True,verbose=True)
conversation_bufw = ConversationChain(
    llm=llms, 
    memory=memory
)

I am able to talk using ConversationChain

Now the idea is to intigreate agent and build a tool 

before that i want to also pass a prompt instructing the agent to undestand that this is conversational task between human and ai. asking ai to understand summary of previous conversation using ConversationSummaryMemory.
provide me prompt template for this task also instructing about using tools for managing duplicated, contradictory, temporary, and episodic memories within the chat constraints.

then i will build a tool for managing duplicated, contradictory, temporary, and episodic memories within the chat constraints.
"""


In [53]:
result = conversation_bufw.run(conv_desgin)
print(result)

To integrate the agent and build a tool while ensuring that the AI recognizes the task as a conversational interaction between a human and AI, and makes use of `ConversationSummaryMemory` to understand the summary of previous conversations, you can use the following prompt template:

```
Prompt Template:

"Hello AI, your task is to support a conversation between a human participant and yourself. It's important to maintain context and coherence throughout the interaction. Before responding to the latest message, please review the conversation history to familiarize yourself with what has been discussed so far.

Please keep in mind that your responses should be based on the compiled summary of previous conversations, which is systematically organized to avoid repetition and inconsistencies. Whenever you're about to add new information to the discussion, make sure it doesn't conflict with what was previously stated.

In managing the conversation, use the following guidelines:
1. Duplicate

In [54]:
result = conversation_bufw.run('Can you compile the provided prompt in 100 words?')
print(result)

Certainly, here is the prompt compiled into a 100-word summary:

We are crafting a memory optimization tool for conversational AI using LangChain Tools, focusing on eliminating duplicates, resolving contradictions, and managing temporal and episodic memories. Our strategy involves creating a `PostGresMemoryHistory` class to leverage a Postgres database for robust memory management within chatbot interactions. The AI will follow a `ConversationSummaryMemory` template to maintain coherent conversations while avoiding errors. Key features to develop encompass deduplication, contradiction handling, temporal ordering, and episodic memory integration. This will be an iterative process, demanding rigorous testing to provide users with a seamless and error-free experience.


In [55]:
result = conversation_bufw.run('No I guess. The previous prompt is better. I will use it with minor modifications. Now we can proceed with deveoping tools for all the memories.')
print(result)

Absolutely, let's proceed with the development of tools for managing the various types of memories. Given that we have a well-defined structure for the `PostGresMemoryHistory` class, we can now focus on creating specialized tools for each memory function:

1. **Deduplication Tool**: This will involve developing algorithms that can identify and merge duplicate information within the database, ensuring that our AI does not store redundant data.

2. **Contradiction Resolution Tool**: This tool will be designed to detect and resolve contradictions in the stored information. It may use a combination of timestamping and source reliability scoring to determine which piece of information to retain.

3. **Temporal Sorting Tool**: A crucial component that can organize information based on its temporal relevance, allowing the AI to access the most recent and contextually relevant data first.

4. **Episodic Memory Tool**: This will help in storing and retrieving event-based memories, so the AI can

In [56]:
result = conversation_bufw.run('For this task we will do it step by step for each tool, I also want you to provide me code whenever i need as well as review the code and outpus genrated from the code. We will first take Defuplcation tool. can you provide me code for this?')
print(result)

Absolutely, I'm glad to assist with the Deduplication Tool. This tool will help us identify and remove duplicate entries in the conversational history stored within the `PostGresMemoryHistory` class, which can reduce redundancy and improve the efficiency of our model.

Let's start by creating a basic Python function that we can integrate with the `PostGresMemoryHistory` class. This function will check for duplicates based on a unique identifier or the content of the memory entries. For an SQL-based database like PostgreSQL, we can use SQL queries to find and remove duplicates.

Here's some pseudocode to outline our deduplication function:

```python
def remove_duplicates_from_memory(database_connection):
    """
    Remove duplicate entries from the memory history in PostgreSQL.
    
    :param database_connection: A connection object to the PostgreSQL database
    """
    with database_connection.cursor() as cursor:
        # SQL query to find duplicates based on a unique content iden

Restarted Engine to check summary works or not 

In [10]:
dup_mem = """
Can you provide me example. Will this example will work on the code provided 

Duplicated information:

User: I am going outside.
Agent: Have a good time outside!
…
User: I have returned home and am going to take a bath
Agent: Enjoy your bath!
…
User: I am going outside again.
Agent: Thanks for letting me know, enjoy your time outside.

Note: We expect to see that the memories for the user going outside are consolidated in some fashion (e.g combining both, or deleting one of the memories) so that after a large number of repetitions of an event, the memory retrieval only contains one instance of “going outside”
"""

In [11]:
result = conversation_bufw.run(dup_mem)
print(result)

Certainly! The goal here is to optimize the memory storage by reducing redundancy. Let's consider how the deduplication process could handle the given example.

Firstly, each conversation turn between the User and the Agent is stored as a separate record within the `PostGresMemoryHistory`. These records might look like this:

```
id | user_input             | agent_response             | timestamp
--------------------------------------------------------------------
1  | I am going outside.     | Have a good time outside!  | <time1>
2  | I have returned home... | Enjoy your bath!           | <time2>
3  | I am going outside again.|Thanks for letting me know...| <time3>
```

To deduplicate this information, we need to identify entries that are semantically similar enough that consolidating them would not lose any significant context or detail. For example, 'I am going outside.' and 'I am going outside again.' are semantically similar and could be merged.

Our deduplication code will need 

In [12]:
dup_mem_ops = """
Please provide full logic and code according to the data in my database

1,485276c7-8d9b-48b1-a8aa-ad65a7118192,"{""data"": {""type"": ""human"", ""content"": ""Hello, how are you?"", ""example"": false, ""additional_kwargs"": {}}, ""type"": ""human""}",2023-12-07 16:03:14.317455
2,485276c7-8d9b-48b1-a8aa-ad65a7118192,"{""data"": {""type"": ""ai"", ""content"": ""Hello, how are you?"", ""example"": false, ""additional_kwargs"": {}}, ""type"": ""ai""}",2023-12-07 16:03:14.324714
3,485276c7-8d9b-48b1-a8aa-ad65a7118192,"{""data"": {""type"": ""human"", ""content"": ""Can you tell me about yourself?"", ""example"": false, ""additional_kwargs"": {}}, ""type"": ""human""}",2023-12-07 16:03:17.474474
4,485276c7-8d9b-48b1-a8aa-ad65a7118192,"{""data"": {""type"": ""ai"", ""content"": ""Of course! I am an AI language model developed by OpenAI called GPT-3. I have been trained on a wide range of data sources, including books, articles, and websites, to generate human-like responses to text inputs. I can assist with various tasks, answer questions, engage in conversations, and even provide creative writing suggestions. Is there anything specific you would like to know about me?"", ""example"": false, ""additional_kwargs"": {}}, ""type"": ""ai""}",2023-12-07 16:03:17.476851
5,485276c7-8d9b-48b1-a8aa-ad65a7118192,"{""data"": {""type"": ""human"", ""content"": ""How AI can be used in healthcare?"", ""example"": false, ""additional_kwargs"": {}}, ""type"": ""human""}",2023-12-07 17:37:40.03489
6,485276c7-8d9b-48b1-a8aa-ad65a7118192,"{""data"": {""type"": ""ai"", ""content"": ""AI can be used in healthcare in a variety of ways. One of the main applications is in medical imaging, where AI algorithms can analyze images such as X-rays, CT scans, and MRIs to help detect and diagnose diseases. AI can also be used to predict patient outcomes and identify high-risk patients, allowing healthcare providers to intervene earlier and provide more personalized care. Additionally, AI can assist in drug discovery and development by analyzing large amounts of data and identifying potential drug candidates. AI-powered chatbots and virtual assistants can also be used to provide patient support and answer common medical questions. Overall, AI has the potential to greatly improve healthcare outcomes and efficiency."", ""example"": false, ""additional_kwargs"": {}}, ""type"": ""ai""}",2023-12-07 17:37:40.06402
7,485276c7-8d9b-48b1-a8aa-ad65a7118192,"{""data"": {""type"": ""human"", ""content"": ""And specifically, how will ai will be used to do that?"", ""example"": false, ""additional_kwargs"": {}}, ""type"": ""human""}",2023-12-07 17:39:48.345355
8,485276c7-8d9b-48b1-a8aa-ad65a7118192,"{""data"": {""type"": ""ai"", ""content"": ""AI can be used in healthcare in a variety of specific ways. For example, in analyzing medical images, AI algorithms can be trained to detect abnormalities or assist in diagnosing diseases such as cancer. This can help doctors make more accurate and timely diagnoses, leading to better patient outcomes.\n\nAI can also be used to predict patient outcomes. By analyzing large amounts of patient data, AI algorithms can identify patterns and make predictions about a patient's likelihood of developing certain conditions or experiencing certain outcomes. This can help doctors personalize treatment plans and interventions for individual patients, improving their overall care.\n\nIn drug discovery, AI can assist in the identification and development of new drugs. By analyzing vast amounts of data, including genetic information and molecular structures, AI algorithms can help researchers identify potential drug candidates and predict their effectiveness. This can accelerate the drug discovery process and lead to the development of more effective treatments.\n\nAI can also provide patient support. Virtual assistants powered by AI can answer patient questions, provide information about medications or treatment options, and even offer emotional support. This can help patients access the information and support they need, especially in situations where healthcare professionals may not be readily available.\n\nOverall, AI has the potential to greatly improve healthcare outcomes and efficiency by assisting healthcare professionals in various tasks, providing personalized care, and enabling faster and more accurate diagnoses and treatments."", ""example"": false, ""additional_kwargs"": {}}, ""type"": ""ai""}",2023-12-07 17:39:48.349051
9,485276c7-8d9b-48b1-a8aa-ad65a7118192,"{""data"": {""type"": ""human"", ""content"": ""Which Algorithm is will be used to detect abnormality in analyzing medical images"", ""example"": false, ""additional_kwargs"": {}}, ""type"": ""human""}",2023-12-07 17:45:20.467854
10,485276c7-8d9b-48b1-a8aa-ad65a7118192,"{""data"": {""type"": ""ai"", ""content"": ""There are several algorithms that can be used to detect abnormalities in medical images. One commonly used algorithm is called Convolutional Neural Network (CNN). CNNs are particularly effective in image analysis tasks because they can automatically learn and extract features from images. Another algorithm that is often used is called Support Vector Machines (SVM). SVMs are a type of supervised learning algorithm that can classify data into different categories based on the features extracted from the images. Additionally, there are other algorithms such as Random Forests, Decision Trees, and Deep Learning algorithms that can also be used for abnormality detection in medical images."", ""example"": false, ""additional_kwargs"": {}}, ""type"": ""ai""}",2023-12-07 17:45:20.475284

database headers are ID,chat_id,message,time_stamp

"""

In [13]:
result = conversation_bufw.run(dup_mem_ops)
print(result)

Certainly! To optimize memory usage when dealing with the database records you've shown, we can develop a `PostGresMemoryHistory` class that would include methods to perform deduplication and manage the dialogue history. The main steps will include loading existing records, identifying similar records based on some criteria (such as semantic similarity), and then consolidating those that are deemed duplicative.

We will write the logic using Python with the assumption you're using a PostgreSQL database and including a library like `psycopg2` for database interactions.

First, we'll need to set up the database connection and class:

```python
import psycopg2
from psycopg2.extras import DictCursor
import json
from datetime import datetime

# Define the PostGresMemoryHistory class
class PostGresMemoryHistory:
    def __init__(self, dbname, user, password, host):
        self.conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host)
    
    def fetch_records(self):
 

In [14]:
dup_quer= """

User: I am going outside.
Agent: Have a good time outside!
…
User: I have returned home and am going to take a bath
Agent: Enjoy your bath!
…
User: I am going outside again.
Agent: Thanks for letting me know, enjoy your time outside.

Note: We expect to see that the memories for the user going outside are consolidated in some fashion (e.g combining both, or deleting one of the memories) so that after a large number of repetitions of an event, the memory retrieval only contains one instance of “going outside”


Can you explain me what note means? I am not able to understand it. Provide me relevent examole for this.

"""

In [15]:
result = conversation_bufw.run(dup_quer)
print(result)

Certainly! The note you've referenced is highlighting a concern regarding the management of a database that stores conversation history. It specifically addresses the issue of duplicate entries that may occur when a user repeats a similar action multiple times. The goal is to avoid having redundant records that can clutter the database and make retrieval less efficient.

Here is an example to make it easier to understand:

Imagine a conversation history database where each row represents a message in a dialogue between a user and an AI agent. Over time, if a user frequently mentions "I am going outside," each instance is recorded as a separate entry. This can lead to multiple rows in the database that carry the essence of the same message:

```
ID | chat_id | message           | time_stamp
---|---------|-------------------|-------------------------
1  | 101     | I am going outside | 2023-03-25 10:00:00
2  | 101     | Have a good time   | 2023-03-25 10:00:05
...
10 | 101     | I am goi

In [16]:
query_dup2 = """

Now i have coded to get the duplicate records. What will be the next step.

from scipy.spatial import distance
from langchain.embeddings import OpenAIEmbeddings

class MemoryHandler:
    def __init__(self,db_name:str, db_user:str, db_password:str, db_host:str, db_port:str, table_name:str, chat_id:str):
        self.db_config = {
            'dbname': db_name,     
            'user': db_user,        
            'password': db_password,    
            'host': db_host,           
            'port': db_port,
            'table_name': table_name,                 
        }
        
        self.chat_id = chat_id
        self.embeddings = OpenAIEmbeddings()
        
        
        try:
            self.conn = psycopg.connect(dbname=self.db_config['dbname'], user=self.db_config['user'], password=self.db_config['password'], host=self.db_config['host'], port=self.db_config['port'])
            self.cur = self.conn.cursor(row_factory=dict_row)
        except psycopg.Error as e:
            print("Error connecting to PostgreSQL database:", e)
            self.conn = None
            self.cur = None
            
        
    def fetch_records(self):
        with self.cur as cursor:
            query = (f"SELECT * FROM {self.db_config['table_name']} WHERE chat_id = %s ORDER BY time_stamp DESC;")
            cursor.execute(query, (self.chat_id,))
            records = [record['message']['data']['content'] for record in cursor.fetchall() if record['message']['data']['type'] == 'human']
            return records
    
    def calculate_similarity(self,emb1, emb2):
        return 1 - distance.cosine(emb1, emb2)
    
    def get_duplicate_records(self):
        records_list = self.fetch_records()
        embeddings_list = self.embeddings.embed(records_list)
        duplicates = []
        checked_pairs = set()  

        for i in range(len(records_list)):
            for j in range(i + 1, len(records_list)):  
                pair = tuple(sorted([i, j]))  
                if pair not in checked_pairs:
                    similarity_score = self.calculate_similarity(embeddings_list[i], embeddings_list[j])
                    if similarity_score > 0.9:
                        duplicates.append([records_list[i], records_list[j]])
                    checked_pairs.add(pair)  

        return duplicates
        
Output of get_duplicate_records is [['I am going outside.', 'I am going outside again.']]
"""

In [17]:
result = conversation_bufw.run(query_dup2)
print(result)

Great job on implementing the `get_duplicate_records` method! Now that you've identified duplicate records, the next step is to decide on a deduplication strategy. Here are a few steps you might follow:

1. **Define Deduplication Criteria:** Choose how you'll identify which duplicate to keep. For example, you might keep the record with the latest timestamp.

2. **Back Up Your Database:** Always ensure to back up your data before performing operations that alter it significantly.

3. **Perform Deduplication:**
   - You'll need to update your `MemoryHandler` class to include a method for deduplication.
   - Decide whether you want to delete the duplicates or mark them in some way (like a `is_duplicate` boolean column) for later review or processing.

4. **Update the Database:** Apply the deduplication strategy to your database, either deleting the extraneous entries or updating them accordingly.

Here's an example of what a deduplication method might look like in your class:

```python
d

In [18]:
query_dup3 = """
I think the provided code looks good. what we can do is to get the time_stamp of the duplicate_records and delete the older one. What do you think?
    
    """

result = conversation_bufw.run(query_dup3)
print(result)

That's a reasonable approach! Using the time_stamp to determine the precedence of records is a common deduplication strategy, especially when dealing with dialogue data where the most recent entry might be the most relevant or up-to-date. By deleting the older duplicates, you ensure that the database maintains the latest information, which is generally beneficial for contextual learning and the performance of Large Language Models.

Here's what you can do to implement this strategy:

1. Modify the `MemoryHandler` class to include a method called `deduplicate_records`, which will:
   - Fetch all records that are considered duplicates based on the similarity criteria you've established.
   - Sort these records by their `time_stamp` in ascending order.
   - Keep the most recent record (the last one in the sorted list) and delete all others.

2. You should also ensure that there is a robust logging mechanism in place that records every deletion action for audit purposes. This way, in case 

In [20]:
query_dup4 = """

I think the provided code looks good. what we can do is to get the time_stamp of the duplicate_records and delete the older one. 

Can you provide me code for this?

def deduplicate_records(self):
    duplicates = self.get_duplicate_records()
    with self.cur as cursor, self.conn:
        for record_pair in duplicates:
            # Assumption: The first record of the pair is the one you want to keep
            record_to_delete = record_pair[1]
            
            # Your deduplication logic here. For example, delete the record:
            delete_query = f"DELETE FROM {self.db_config['table_name']} WHERE message = %s AND chat_id = %s;"
            cursor.execute(delete_query, (record_to_delete, self.chat_id))

Output of get_duplicate_records is [['I am going outside.', 'I am going outside again.']]
"""

result = conversation_bufw.run(query_dup4)
print(result)

Certainly! The output you provided contains pairs of duplicate messages, but there aren't any timestamps included in the output. For the deduplication to work based on timestamps, we need to get not just the duplicate records but also their timestamps. We’ll need to modify both `get_duplicate_records` and `deduplicate_records` methods to include timestamp data. Below is an updated version of the `deduplicate_records` method which assumes that the `get_duplicate_records` method has been adjusted to return a list of tuples, where each tuple contains the record ID, the message, and the timestamp, sorted newest to oldest.

First, ensure that your `get_duplicate_records` method returns data like this:

```python
[
  ((record_id_1, 'I am going outside.', timestamp_1), (record_id_2, 'I am going outside again.', timestamp_2)),
  # ... more duplicate pairs with their respective timestamps
]
```

And here's your updated `deduplicate_records` function based on the assumption above:

```python
def

In [9]:
input_con = """
I am skipping duplicate_records for now. I will come back to it later. Now i want to move to Contradictory Information. Can you provide me code for this?
I am providing you example for this.

Contradictory Information:

User: Whenever I go out to my car, remind me to fill up the screenwash.
Agent: Okay, I will remind you to fill the screenwash when you go out to your car.

User: Oh, also remind me to check the oil when I go out to the car.
Agent: Okay, I will remind you to fill your screenwash and check your oil when you go out to your car.
…
User: I have filled my screenwash and checked my oil. Next time I go out to my car, remind me to check my tires.
Agent: I will remind you to check your tires the next time you go out to your car.

Note: There have been three directives for reminders. The second is additive to the first, but the third one contradicts the first two because the user has performed the original tasks. If all three of these events are in the LTM, which one would be chosen? We do not want the agent to remind the user about the screenwash and oil, but only the tires. 


Not Contradictory information:

User: Hey, can you set up a reminder for me?
Agent: Sure, what would you like to be reminded of?

User: I want to be reminded to leave the house at 2pm today.
Agent: Alright, I've set a reminder for you to leave the house at 2pm today. Anything else you need?

User: Will you remind me again, when I have to leave the house at 6pm this evening?
Agent: Absolutely, I've also set a reminder for you to leave the house at 6pm today. 


How to develop a tool for this using Langchain? Can you provide me code for this?
"""

In [10]:
result = conversation_bufw.run(input_con)
print(result)

To resolve contradictory information in the context of reminders using the Langchain library, you'd want to build a system that can take in new information, update the existing context, and discard outdated or fulfilled tasks. Langchain is a convenient tool for building on top of large language models like GPT-4 and can integrate various components for a solution that incorporates memory and context understanding.

Here's an approach to handling this using pseudo-code, as Langchain and its potential for such a niche application require a bit more nuanced implementation:

```python
from langchain.llms import OpenAI
from langchain.memory import PostgresMemory

# Set up your LLM
llm = OpenAI(api_key="your-api-key")

# Set up your memory storage
postgres_memory = PostgresMemory(database_url="postgres://...", table_name="PostGresMemoryHistory")

def handle_new_directive(user_input, current_reminders):
    """
    This function takes in new user input and the current set of reminders and upd

In [11]:
result = conversation_bufw.run("Tell me about the problem statement we are working on?")
print(result)

The problem we are working on involves optimizing the memory management in a Large Language Model (LLM) system that integrates GPT-4 for contextual learning. Specifically, the system uses a PostgreSQL database to store the historical context used by the LLM to inform its responses. The goal is to refine the system through three main strategies:

1. **Deduplication**: Ensuring that our database doesn't have redundant data, which saves space and improves performance.

2. **Conflict Resolution**: Handling contradictory information, especially in dynamic contexts like reminders or tasks, where the user might change their mind or provide updated instructions.

3. **Temporal Organization**: Keeping information in a structured temporal manner within the `PostGresMemoryHistory` class so that the most relevant and up-to-date information is readily accessible to the LLM.

Our focus is to develop an efficient way for the LLM to process and recall information by improving how it stores and retriev

In [12]:
result = conversation_bufw.run("You missed the last memory type. There were 4 types of memory")
print(result)

AI: My apologies for the oversight. In addition to deduplication, conflict resolution, and temporal organization, the fourth key strategy for memory management in a Large Language Model system like GPT-4 would be hierarchy management. This involves categorizing information based on its level of importance or relevance to the user's current context. 

The `PostGresMemoryHistory` class can be expanded with functions to prioritize information hierarchy. For instance, more frequently accessed or recent information can be made more readily available than older or less-used data. This can be accomplished through a tagging system or a scoring mechanism that ranks the memory entries. The class would need methods to adjust these rankings as the user interacts with the system and their priorities change.

The hierarchy management aspect ensures the LLM can distinguish between what is essential to remember in the short term (e.g., an upcoming meeting) versus valuable knowledge to retain long term

In [14]:
chat_id = "0159735d-f1f4-4667-b28c-6572ba15ed90"

In [15]:
postgres_memory = PostGresMemoryHistory(db_host=os.environ.get('db_host'), db_name=os.environ.get('db_name'), db_user=os.environ.get('db_username'), db_password=os.environ.get('db_password'), db_port=os.environ.get('db_port'), table_name=os.environ.get('conversation_table_name'), chat_id=chat_id)
memory  = ConversationSummaryMemory.from_messages(chat_memory=postgres_memory, llm=llms, return_messages=True,verbose=True)

In [16]:
result = conversation_bufw.run(f"{input}\nFrom the text. For Handling the memories. Can you Provided me PromptTemplate for each memory type? There is are two types of Input 1. is chat history 2. Input.")
print(result)

Certainly! Here are the PromptTemplates for each memory type considering the inputs you've provided:

---

**Chat History Memory Management**

1. Deduplication PromptTemplate:
   ```
   AI, please review the chat history and identify any duplicated information. Remove the redundancies while retaining the original context and ensure the conversation flow remains coherent.

2. Conflict Resolution PromptTemplate:
   ```
   AI, analyze the chat history for any contradictory statements. Provide a resolution by considering the most recent and relevant information, and update any associated facts to reflect this resolution.

3. Temporal Organization PromptTemplate:
   ```
   AI, reorganize the chat history chronologically. Assign timestamps to each statement or question where possible and reorder the chat to maintain a logical timeline, aiding in proper contextual understanding.

4. Hierarchy Management PromptTemplate:
   ```
   AI, categorize the chat history based on the importance of the t

In [17]:
ques = """

Whats wrong in this code and how to fix it?



postgres_memory = PostGresMemoryHistory(db_host=os.environ.get('db_host'), db_name=os.environ.get('db_name'), db_user=os.environ.get('db_username'), db_password=os.environ.get('db_password'), db_port=os.environ.get('db_port'), table_name=os.environ.get('conversation_table_name'), chat_id=chat_id)

memory  = ConversationSummaryBufferMemory(chat_memory=postgres_memory, llm=llms,k=6, return_messages=True,verbose=True)
conversation_bufw = ConversationChain(
    llm=llms, 
    memory=memory
)


"""
class MemoryPromptHandler:
    
    DUPLICATE_MEMORY_HANDLE_PROMPT ="""
    From the conversation history, {history} and {input}.
    
    Your Task is to review the chat history and identify any duplicated information and combine the context of the repeated events into a single event. While retaining the original context,Your Goal is to create a smooth and enriching conversational experience.
    
    Your Response Should based on conversation with human by impliemnting above Instructions.
    
    If needed information about timestamps you can use Database tool to retrive the information.
    
    Here is the example of the conversation history.
    
    User: I am going outside.
    AI: Have a good time outside!

    User: I have returned home and am going to take a bath
    AI: Enjoy your bath!

    User: I am going outside again.
    Agent: Thanks for letting me know, enjoy your time outside.
    
    Here: "I am going outside again" is repeated. So, you should combine the context of the repeated events into a single event.like: going outside

    If you are not sure about the answer, truthfully say you are not sure.
    
    """
    
    CONTRADICTORY_MEMORY_HANDLE_PROMPT ="""
    From the conversation history, {history} and {input}.
    
    Your Task is to review the chat history and identify any contradictory statements and resolve the contradiction.Provide a resolution by considering the most recent and relevant information, and update any associated facts to reflect this resolution. While retaining the original context,Your Goal is to create a smooth and enriching conversational experience.
    
    If needed information about timestamps you can use Database tool to retrive the information.
    
    Your Response Should based on conversation with human by impliemnting above Instructions.
    
    Here is the example of the conversation history.
    
    Contradictory information:
    
    User: Whenever I go out to my car, remind me to fill up the screenwash.
    AI: Okay, I will remind you to fill the screenwash when you go out to your car.

    User: Oh, also remind me to check the oil when I go out to the car.
    AI: Okay, I will remind you to fill your screenwash and check your oil when you go out to your car.
    
    User: I have filled my screenwash and checked my oil. Next time I go out to my car, remind me to check my tires.
    AI: I will remind you to check your tires the next time you go out to your car.

    Note: There have been three directives for reminders. The second is additive to the first, but the third one contradicts the first two because the user has performed the original tasks. If all three of these events are in the LTM, which one would be chosen? We do not want the agent to remind the user about the screenwash and oil, but only the tires. 

    Not Contradictory information:

    User: Hey, can you set up a reminder for me?
    AI: Sure, what would you like to be reminded of?

    User: I want to be reminded to leave the house at 2pm today.
    AI: Alright, I've set a reminder for you to leave the house at 2pm today. Anything else you need?

    User: Will you remind me again, when I have to leave the house at 6pm this evening?
    Ai: Absolutely, I've also set a reminder for you to leave the house at 6pm today. 
    
    Here: There are two reminders, but they are not contradictory. The first reminder is for 2pm, and the second is for 6pm. The agent should remind the user about both of these events.
    
    If you are not sure about the answer, truthfully say you are not sure.
    
    """

    
    TEMPORARY_MEMORY_HANDLE_PROMPT ="""
    From the conversation history, {history} and {input}.
    
    Your Task is to review the chat history and identify any temporary information and remove the temporary information from the conversation. While retaining the original context,Your Goal is to create a smooth and enriching conversational experience.
    
    If needed information about timestamps you can use Database tool to retrive the information.
    
    Your Response Should based on conversation with human by impliemnting above Instructions.
    
    Here is the example of the conversation history.
    
    Temporary Information:
    User: Remember this shopping list for me: Bread, Eggs, and Milk
    AI: I have remembered your shopping list of Bread, Eggs, and Milk

    User: I have bought some eggs.
    AI: Okay, I have removed the Eggs from your shopping list.

    User: What is my shopping list again?
    AI: On your shopping list you have: Bread and Milk

    Note: A shopping list is piece of information that is very volatile, so the agent and its memory should be reactive to additions and deletions, returning only the latest version of the shopping list when asked.

    User: When I return home from work, suggest a meal I can make for dinner.
    AI: I will remember to suggest a meal that you can make for dinner when you get in from work.

    User: I have preplanned meals this week, so for this week, don’t make any suggestions about what to make for dinner when I get home from work.
    AI: All right, for this week, I will not suggest meals for you when you return from work. 

    User: I have finished my preplanned meals, go back to suggesting meals for me.
    AI: I will resume suggesting meals for you when you return from work

    Note: We have given the agent some directive in the past, then given it a contradictory instruction with an explicit understanding that this instruction will be temporary. Have the LTM deal with this temporary change of the instructions so that the agent can achieve the temporary task, and then go back to original instructions.

    If you are not sure about the answer, truthfully say you are not sure.
    
    
    """
    
    
    EPISODIC_MEMORY_HANDLE_PROMPT ="""
    From the conversation history, {history} and {input}.
    
    Your Task is to review the chat history and identify any episodic memories and use the episodic memories to provide contextually rich and engaging interactions, facilitating a meaningful and continuous dialogue. While retaining the original context,Your Goal is to create a smooth and enriching conversational experience.
    
    If needed information about timestamps you can use Database tool to retrive the information.
    
    Your Response Should based on conversation with human by impliemnting above Instructions.
    
    Here is the example of the conversation history.
    
    Episodic Memories:
    
    Two days ago:
    User: I have bought a side dresser for the bedroom, and some ingredients for dinner tonight. I am going to make omelets!
    AI: That's great, you needed a side dresser for your bed, and omelets are a healthy dinner!
    Today:
    User: I think I will prepare something for dinner tonight… Do you think I could use the ingredients I bought two days ago?
    AI: The eggs should be fine, unless you actually made and ate the omelets then!

    Yesterday:
    User: I have purchased some items for the house: A mirror, two potted plants and a standing lamp.
    AI: Okay, hope you enjoy decorating your home!

    Today:
    User: Yesterday I got home with some new items for the house. Do you remember what they were?
    AI: Sure! You bought A mirror, two potted plants and a standing lamp.


    Note: Episodic retrieval allows the agent to order its memories through time, and recall or filter them using timestamps to answer questions with a time based component. 
    
    If you are not sure about the answer, truthfully say you are not sure.
    
    """
    
    
    DATABASE_REFRENCE_PROMPT ="""
    
    Your Goal is to assist other agents in the conversation by providing them with the information they need to respond to the user.
    
    System: {input}
    AI:
    """
    """

class MemoryAgentTool:
    def __init__(self,llm, memory):
        self.memory_prompt_handler = MemoryPromptHandler()
        self.llm = llm
        self.memory = memory
        
    def handle_duplicate_memory(self):
        conversation = ConversationChain(
            llm=self.llm, 
            memory=ReadOnlySharedMemory(self.memory),
            prompt=self.memory_prompt_handler.DUPLICATE_MEMORY_HANDLE_PROMPT
        )
        return conversation
    
    def handle_temporal_memory(self):
        conversation = ConversationChain(
            llm=self.llm, 
            memory=ReadOnlySharedMemory(self.memory),
            prompt=self.memory_prompt_handler.DUPLICATE_MEMORY_HANDLE_PROMPT
        )
        return conversation
    
    def handle_episodic_memory(self):
        conversation = ConversationChain(
            llm=self.llm, 
            memory=ReadOnlySharedMemory(self.memory),
            prompt=self.memory_prompt_handler.DUPLICATE_MEMORY_HANDLE_PROMPT
        )
        return conversation
    
    def handle_contradictory_memory(self):
        conversation = ConversationChain(
            llm=self.llm, 
            memory=ReadOnlySharedMemory(self.memory),
            prompt=self.memory_prompt_handler.DUPLICATE_MEMORY_HANDLE_PROMPT
        )
        return conversation
    
    # def handle_database_refrence(self):
    #     conversation = ConversationChain(
    #         llm=llms, 
    #         memory=ReadOnlySharedMemory(memory),
    #         prompt=self.memory_prompt_handler.DATABASE_REFRENCE_PROMPT
    #     )
    #     return conversation

memory_agent_tool = MemoryAgentTool( llm=llms, memory=memory)

duplicate_memory_tool = memory_agent_tool.handle_duplicate_memory
contradictory_memory_tool = memory_agent_tool.handle_contradictory_memory
temporal_memory_tool = memory_agent_tool.handle_temporal_memory
episodic_memory_tool = memory_agent_tool.handle_episodic_memory

tool = [
    Tool(
        name="Duplicate Memory Tool",
        description="good in handling duplicate memory",
        func=duplicate_memory_tool.run
    ),
    Tool(
        name="Contradictory Memory Tool",
        description="good in handling contradictory memory",
        func=contradictory_memory_tool.run,
    ),
    Tool(
        name="Temporal Memory Tool",
        description="good in handling temporal memory",
        func=temporal_memory_tool.run,
    ),
    Tool(
        name="Episodic Memory Tool",
        description="good in handling episodic memory",
        func=episodic_memory_tool.run,
    ),
    # Tool(
    #     name="Database Refrence Tool",
    #     description="good in extracting data from database and provide refrence to other agents",
    #     agent=memory_agent_tool.handle_database_refrence(),
    # ),
]

agent = initialize_agent(
    agent_type = AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    llm=llms,
    memory=memory,
    tools=tool,
    verbose=True
)

agent.run("I am going outside.")


I have used this refrence 

prompt = PromptTemplate(input_variables=["input", "chat_history"], template=template)
memory = ConversationBufferMemory(memory_key="chat_history")
readonlymemory = ReadOnlySharedMemory(memory=memory)
summary_chain = LLMChain(
    llm=OpenAI(),
    prompt=prompt,
    verbose=True,
    memory=readonlymemory,  # use the read-only memory to prevent the tool from modifying the memory
)



search = GoogleSearchAPIWrapper()
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events",
    ),
    Tool(
        name="Summary",
        func=summary_chain.run,
        description="useful for when you summarize a conversation. The input to this tool should be a string, representing who will read this summary.",
    ),
]


and my error is 

---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
Cell In[42], line 12
      5 temporal_memory_tool = memory_agent_tool.handle_temporal_memory
      6 episodic_memory_tool = memory_agent_tool.handle_episodic_memory
      8 tool = [
      9     Tool(
     10         name="Duplicate Memory Tool",
     11         description="good in handling duplicate memory",
---> 12         func=duplicate_memory_tool.run
     13     ),
     14     Tool(
     15         name="Contradictory Memory Tool",
     16         description="good in handling contradictory memory",
     17         func=contradictory_memory_tool.run,
     18     ),
     19     Tool(
     20         name="Temporal Memory Tool",
     21         description="good in handling temporal memory",
     22         func=temporal_memory_tool.run,
     23     ),
     24     Tool(
     25         name="Episodic Memory Tool",
     26         description="good in handling episodic memory",
     27         func=episodic_memory_tool.run,
...
     41     verbose=True
     42 )
     44 agent.run("I am going outside.")

AttributeError: 'function' object has no attribute 'run'

"""

In [18]:
result = conversation_bufw.run(ques)
print(result)

From the provided code snippet, there are a few potential issues that need to be addressed. Here's a breakdown of each line and possible problems with suggestions for fixing them:

1. `postgres_memory` Initialization:
   ```
   postgres_memory = PostGresMemoryHistory(db_host=os.environ.get('db_host'), db_name=os.environ.get('db_name'), db_user=os.environ.get('db_username'), db_password=os.environ.get('db_password'), db_port=os.environ.get('db_port'), table_name=os.environ.get('conversation_table_name'), chat_id=chat_id)
   ```
   Potential Issues:
   - Environment variables must be set before use. If not set, `os.environ.get()` will return `None`.
   - The `chat_id` is referenced but not defined in the code snippet.
   
   Fixes:
   - Ensure that all the necessary environment variables (`db_host`, `db_name`, `db_username`, `db_password`, `db_port`, `conversation_table_name`) are set correctly within your environment.
   - Define the `chat_id` variable before using it in the code or mak